## This is a notebook that verify the communication of the pointing kernel with the hexapods (including the rotator and Mount)
It is part of a series of notebook that tests the communication between the pointing kernel and different components:
- the mount
- the rotator
- M1M3
- M2
- Cam hexapod
- M2 hexapod
We will leave ComCam out of this notebook

The goals of the exercice are 
- Check heartbeat from all components.
- Make sure we can put all components in ENABLE state and back to STANDBY.
- Check the EFD to ensure that the correct events/commands/telemetry is published

This will ensure that we are ready to check the slew when needed


Library loading

In [ ]:
#from lsst.ts.observatory.maintel import MTCS
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

import asyncio

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = True

from datetime import datetime


## Set up -> things that would go into the tcs script

In [ ]:
now = datetime.now()

In [ ]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
#add a sleep right now (january 2021)
await asyncio.sleep(10.)

mtptg = salobj.Remote(script.domain, "MTPtg")
rot = salobj.Remote(script.domain, "MTRotator")
print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
mtm = salobj.Remote(script.domain, "NewMTMount")

In [ ]:
camHex = salobj.Remote(script.domain, "MTHexapod", index = '1')
m2Hex = salobj.Remote(script.domain, "MTHexapod", index = '2')
#print(f'time to start is {datetime.now() - start_time} [s]')

In [ ]:
await asyncio.gather(mtptg.start_task,
                     script.start_task,
                     rot.start_task,
                     camHex.start_task,
                     m2Hex.start_task)

In [ ]:
await mtptg.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
await rot.evt_heartbeat.next(flush=True, timeout=5)
await camHex.evt_heartbeat.next(flush=True, timeout=5)
await m2Hex.evt_heartbeat.next(flush=True, timeout=5)

In [ ]:
summary_state =  await mtptg.evt_summaryState.aget(timeout=5)
print(salobj.State(summary_state.summaryState))

#await mtptg.cmd_start.start(timeout=5)
I don't need it? depends… if the mtptg is enable, then you don’t need it
you can replace that one with
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)
which will make sure the CSC is in enabled state at the end…

Check the state of the system

In [ ]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(m2Hex, salobj.State.STANDBY)
await salobj.set_summary_state(camHex, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(m2Hex, state = salobj.State.ENABLED)
await salobj.set_summary_state(camHex, state = salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(m2Hex, state = salobj.State.ENABLED, settingsToApply="default")

In [ ]:
await salobj.set_summary_state(camHex, state = salobj.State.ENABLED, settingsToApply="default")

In [ ]:
#Check which hexapod we are controlling
whichHex1 = await camHex.evt_configuration.aget(timeout=10.)
print("Hexapod camHex", whichHex1.MTHexapodID)
print(camHex.salinfo.index)

whichHex2 = await m2Hex.evt_configuration.aget(timeout=10.)
print("Hexapod m2Hex", whichHex2.MTHexapodID)
print(m2Hex.salinfo.index)

In [ ]:
print(whichHex1)

In [ ]:
camHexPosition0 = await camHex.tel_actuators.aget(timeout=10.)
print(camHexPosition0)

In [ ]:
#print("camera Hexapod positions", camHexPosition.MTHexapodID,  camHexPosition.calibrated0,  camHexPosition.calibrated1,  camHexPosition.calibrated2,  camHexPosition.calibrated3,  camHexPosition.calibrated4,  camHexPosition.calibrated5)
print("camera Hexapod positions", camHexPosition0.MTHexapodID,  camHexPosition0.calibrated)

In [ ]:
m2HexPosition0 = await m2Hex.tel_actuators.aget(timeout=10.)
print("M2 Hexapod positions", m2HexPosition0.MTHexapodID,  m2HexPosition0.calibrated)

Enter the location for the telescope for the pointing

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
#Get the elevation and rotator angle at the beginning of a run.


In [ ]:
mtptgDemand0 = await mtptg.tel_currentTargetStatus.aget(timeout=10.)
print("Current target status")
print("Az", mtptgDemand0.demandAz, "El", mtptgDemand0.demandEl, "Rot", mtptgDemand0.demandRot )

In [ ]:
# enable the hexapods LUT compensation
await m2Hex.cmd_setCompensationMode.set_start(enable=1, timeout=10)
await camHex.cmd_setCompensationMode.set_start(enable=1, timeout=10)

In [ ]:
m2hex_mode = await m2Hex.evt_compensationMode.aget(timeout=10)
print("M2 compsensation mode enabled?",m2hex_mode.enabled)
camhex_mode = await camHex.evt_compensationMode.aget(timeout=10)
print("Cam compsensation mode enabled?",camhex_mode.enabled)

## Start of a first slew. Elevation of 80

In [ ]:
now = datetime.now()
print("Start to point the telescope", now)

alt = 70. * u.deg
az = 25. * u.deg
rot_tel = Angle(0, unit= u.deg) 

target_name="TMA motion test"
#getting time for the pointing to ensure that we're using the time that the mount seems it is
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
#Convert the astropy class
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
#Check that we're getting the right time
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")
print(curr_time_mtptg.tai.value)

#create the object with the alt az coordinate + time to be able to convert them in RA and Dec
cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_mtptg.tai, 
                location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)

# Calculating the other parameters      
rot_pa = rot_tel
print(rot_pa)  


In [ ]:
mtptgDemand = await mtptg.tel_currentTargetStatus.aget(timeout=10.)
print("Current target status")
print("Az", mtptgDemand.demandAz, "El", mtptgDemand.demandEl, "Rot", mtptgDemand.demandRot )

In [ ]:
#The pointing component is commanding the mount directly
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)
print("System Ready")

await m2Hex.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True, timeout=10)
await camHex.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True, timeout=10)

In [ ]:
mtptgDemand = await mtptg.tel_currentTargetStatus.aget(timeout=10.)
print("Current target status")
print("Az", mtptgDemand.demandAz, "El", mtptgDemand.demandEl, "Rot", mtptgDemand.demandRot )

In [ ]:
m2hex_mode = await m2Hex.evt_compensationMode.aget(timeout=10)
print("M2 compsensation mode enabled?",m2hex_mode.enabled)
camhex_mode = await camHex.evt_compensationMode.aget(timeout=10)
print("Cam compsensation mode enabled?",camhex_mode.enabled)

## Check telemetry

In [ ]:
print(camHex.salinfo.command_names)

In [ ]:
print(camHex.salinfo.event_names)

In [ ]:
mtptgDemand = await mtptg.tel_currentTargetStatus.aget(timeout=10.)
print("Current target status")
print("Az", mtptgDemand.demandAz, "El", mtptgDemand.demandEl, "Rot", mtptgDemand.demandRot )
ElMat = np.array(mtptgDemand.demandEl)

rotAngle = await rot.tel_rotation.aget()
print(rotAngle.actualPosition)

#print("camera Hexapod positions", camHexPosition.MTHexapodID,  camHexPosition.calibrated0,  camHexPosition.calibrated1,  camHexPosition.calibrated2,  camHexPosition.calibrated3,  camHexPosition.calibrated4,  camHexPosition.calibrated5)
camHexPosition = await camHex.tel_actuators.aget(timeout=10.)
print("camera Hexapod positions", camHexPosition.MTHexapodID,  camHexPosition.calibrated)
camHexPosMat = np.array(camHexPosition.calibrated)

m2HexPosition = await m2Hex.tel_actuators.aget(timeout=10.)
print("M2 Hexapod positions", m2HexPosition.MTHexapodID,  m2HexPosition.calibrated)
m2HexPosMat = np.array(m2HexPosition.calibrated)

#Comparison between the actuator position before and after the move
#m2DiffHexpos = list(set(m2HexPosition.calibrated) - set(m2HexPosition0.calibrated))
#print("M2 Hex motion", m2DiffHexpos)
#camDiffHexpos = list(set(camHexPosition.calibrated) - set(camHexPosition0.calibrated))
#print("Cam Hex motion", camDiffHexpos)

In [ ]:
#check the compensation offset
compensationDataCam = await camHex.evt_compensationOffset.aget(timeout=10)
print("Elevation used by the camera Hexapod", compensationDataCam.elevation)
compensationDataM2 = await m2Hex.evt_compensationOffset.aget(timeout=10)
print("Elevation used by the M2 Hexapod", compensationDataM2.elevation)

In [ ]:
#Check the offsets sent
print("M2 Hex LUT sent",compensationDataM2.x,compensationDataM2.y,compensationDataM2.z,compensationDataM2.u,compensationDataM2.v,compensationDataM2.w)
print("Cam Hex LUT sent",compensationDataCam.x,compensationDataCam.y,compensationDataCam.z,compensationDataCam.u,compensationDataCam.v,compensationDataCam.w)

In [ ]:
camHex_input_position = await camHex.evt_compensatedPosition.aget(timeout=10.)
m2Hex_input_position = await m2Hex.evt_compensatedPosition.aget(timeout=10.)
print("M2 Hex position sent",m2Hex_input_position.x,m2Hex_input_position.y,m2Hex_input_position.z,m2Hex_input_position.u,m2Hex_input_position.v,m2Hex_input_position.w)
print("Camera Hex position sent",camHex_input_position.x,camHex_input_position.y,camHex_input_position.z,camHex_input_position.u,camHex_input_position.v,camHex_input_position.w)

In [ ]:
camHex_input_uncomp_position = await camHex.evt_uncompensatedPosition.aget(timeout=10.)
m2Hex_input_uncomp_position = await m2Hex.evt_uncompensatedPosition.aget(timeout=10.)
print("M2 Hex position sent",m2Hex_input_uncomp_position.x,m2Hex_input_uncomp_position.y,m2Hex_input_uncomp_position.z,m2Hex_input_uncomp_position.u,m2Hex_input_uncomp_position.v,m2Hex_input_uncomp_position.w)
print("Camera Hex position sent",camHex_input_uncomp_position.x,camHex_input_uncomp_position.y,camHex_input_uncomp_position.z,camHex_input_uncomp_position.u,camHex_input_uncomp_position.v,camHex_input_uncomp_position.w)

# Start to loop a few Elevations

In [ ]:
ElInputMat = np.arange(80,10,-10)
print(ElInputMat)

In [ ]:
ElMat = []
camHexPosMat = [] 
m2HexPosMat = [] 
camHex_input_pos_mat = [] 
m2Hex_input_pos_mat = [] 


for El in ElInputMat: 
    now = datetime.now()
    print("Start to point the telescope", now) 
    print("Elevation is", El)

    alt = El * u.deg
    az = 25. * u.deg
    rot_tel = Angle(0, unit= u.deg) 

    target_name="TMA motion test"
    #getting time for the pointing to ensure that we're using the time that the mount seems it is
    time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
    #Convert the astropy class
    curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
    #Check that we're getting the right time
    time_err = curr_time_mtptg - Time.now()
    print(f"Time error={time_err.sec:0.2f} sec")
    print(curr_time_mtptg.tai.value)

    #create the object with the alt az coordinate + time to be able to convert them in RA and Dec
    cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_mtptg.tai, 
                location=location)
    cmd_radec = cmd_elaz.transform_to(ICRS)

    # Calculating the other parameters      
    rot_pa = rot_tel
    print(rot_pa)
    
    #The pointing component is commanding the mount directly
    ack = await mtptg.cmd_raDecTarget.set_start(
        targetName=target_name,
        frame=ATPtg.CoordFrame.ICRS,
        epoch=2000,  # should be ignored: no parallax or proper motion
        equinox=2000,  # should be ignored for ICRS
        ra=cmd_radec.ra.hour,
        declination=cmd_radec.dec.deg,
        parallax=0,
        pmRA=0,
        pmDec=0,
        rv=0,
        dRA=0,
        dDec=0,
        rotPA=rot_pa.deg-180,
        rotFrame=ATPtg.RotFrame.FIXED,
        rotMode=ATPtg.RotMode.FIELD,
        timeout=10
    )

    print("Waiting 10s")
    await asyncio.sleep(10.)
    print("System Ready")
    
    
    ## Make sure the Hexapods take into account the elevation
    await m2Hex.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True, timeout=10)
    await camHex.cmd_move.set_start(x=0,y=0,z=0,u=0,v=0,w=0,sync=True, timeout=10)
    
    ## Check telemetry

    mtptgDemand = await mtptg.tel_currentTargetStatus.aget(timeout=10.)
    print("Current target status")
    print("Az", mtptgDemand.demandAz, "El", mtptgDemand.demandEl, "Rot", mtptgDemand.demandRot )
    ElMat = np.append(ElMat,np.array(mtptgDemand.demandEl))

    #check the compensation offset
    compensationDataCam = await camHex.evt_compensationOffset.aget(timeout=10)
    print("Elevation used by the camera Hexapod", compensationDataCam.elevation)
    compensationDataM2 = await m2Hex.evt_compensationOffset.aget(timeout=10)
    print("Elevation used by the M2 Hexapod", compensationDataM2.elevation)
    
    
    #Evolution of the compensated position as a function of elevation 
    camHex_input_position = await camHex.evt_compensatedPosition.aget(timeout=10.)
    m2Hex_input_position = await m2Hex.evt_compensatedPosition.aget(timeout=10.)
   
    tmp_m2=[m2Hex_input_position.x,m2Hex_input_position.y,m2Hex_input_position.z,m2Hex_input_position.u,m2Hex_input_position.v,m2Hex_input_position.w]
    tmp_cam=[camHex_input_position.x,camHex_input_position.y,camHex_input_position.z,camHex_input_position.u,camHex_input_position.v,camHex_input_position.w] 

 #   tmp_m2=[compensationDataM2.x,compensationDataM2.y,compensationDataM2.z,compensationDataM2.u,compensationDataM2.v,compensationDataM2.w]
 #   tmp_cam=[compensationDataCam.x,compensationDataCam.y,compensationDataCam.z,compensationDataCam.u,compensationDataCam.v,compensationDataCam.w] 
    m2Hex_input_pos_mat = np.append(m2Hex_input_pos_mat,np.array(tmp_m2) )
    camHex_input_pos_mat= np.append(camHex_input_pos_mat, np.array(tmp_cam))
   
   #Evolution of the actuator position as a function of elevation 
    camHexPosition = await camHex.tel_actuators.aget(timeout=10.)
    m2HexPosition = await m2Hex.tel_actuators.aget(timeout=10.)
    
    m2HexPosMat = np.append(m2HexPosMat, np.array(m2HexPosition.calibrated))
    camHexPosMat = np.append(camHexPosMat, np.array(camHexPosition.calibrated))
        
    
print("End of loop")    

## Plot of the evolution as a function of elevation

In [ ]:
m2HexPosMat = np.reshape(m2HexPosMat,(-1,6))
print(m2HexPosMat)
print(ElMat)
plt.plot(ElMat, m2HexPosMat)
plt.show()

In [ ]:
m2Hex_input_pos_mat = np.reshape(m2Hex_input_pos_mat,(-1,6))
#print(m2Hex_input_pos_mat)
print(ElMat)
fig=plt.figure(figsize=(30, 20))
#fig, axs = plt.subplots(2, 3)

ax11 = fig.add_subplot(231)
ax12= fig.add_subplot(232)
ax13= fig.add_subplot(233)
ax21 = fig.add_subplot(234)
ax22= fig.add_subplot(235)
ax23= fig.add_subplot(236)

ax11.plot(ElMat, m2Hex_input_pos_mat[:,0])
ax11.set_title('x')
ax11.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax12.plot(ElMat, m2Hex_input_pos_mat[:,1])
ax12.set_title('y')
ax12.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax13.plot(ElMat, m2Hex_input_pos_mat[:,2])
ax13.set_title('z')
ax13.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax21.plot(ElMat, m2Hex_input_pos_mat[:,3])
ax21.set_title('u')
ax21.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")
ax22.plot(ElMat, m2Hex_input_pos_mat[:,4])
ax22.set_title('v')
ax22.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")
ax23.plot(ElMat, m2Hex_input_pos_mat[:,5])
ax23.set_title('w')
ax23.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")

#plt.plot(ElMat, m2Hex_input_pos_mat)
plt.show()

In [ ]:
camHexPosMat = np.reshape(camHexPosMat,(-1,6))
print(camHexPosMat)
print(ElMat) 
plt.plot(ElMat, camHexPosMat)
plt.show()

In [ ]:
camHex_input_pos_mat = np.reshape(camHex_input_pos_mat,(-1,6))
#print(camHex_input_pos_mat)
print(ElMat)
fig, axs = plt.subplots(2, 3)

#fig, axs = plt.subplots(2, 3, gridspec_kw={ 'width_ratios': [2, 2,1],  'height_ratios': [2, 1]})

axs[0, 0].plot(ElMat, camHex_input_pos_mat[:,0])
axs[0, 0].set_title('x')
axs[0, 1].plot(ElMat, camHex_input_pos_mat[:,1])
axs[0, 1].set_title('y')
axs[0, 2].plot(ElMat, camHex_input_pos_mat[:,2])
axs[0, 2].set_title('z')
axs[1, 0].plot(ElMat, camHex_input_pos_mat[:,3])
axs[1, 0].set_title('u')
axs[1, 1].plot(ElMat, camHex_input_pos_mat[:,4])
axs[1, 1].set_title('v')
axs[1, 2].plot(ElMat, camHex_input_pos_mat[:,5])
axs[1, 2].set_title('w')

#plt.plot(ElMat, m2Hex_input_pos_mat)
plt.show()


camHex_input_pos_mat = np.reshape(camHex_input_pos_mat,(-1,6))
#print(m2Hex_input_pos_mat)
print(ElMat)
fig=plt.figure(figsize=(30, 20))
#fig, axs = plt.subplots(2, 3)

ax11 = fig.add_subplot(231)
ax12= fig.add_subplot(232)
ax13= fig.add_subplot(233)
ax21 = fig.add_subplot(234)
ax22= fig.add_subplot(235)
ax23= fig.add_subplot(236)

ax11.plot(ElMat, camHex_input_pos_mat[:,0])
ax11.set_title('x')
ax11.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax12.plot(ElMat, camHex_input_pos_mat[:,1])
ax12.set_title('y')
ax12.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax13.plot(ElMat, camHex_input_pos_mat[:,2])
ax13.set_title('z')
ax13.set(xlabel='Elevation in deg', ylabel = "Position in microns")
ax21.plot(ElMat, camHex_input_pos_mat[:,3])
ax21.set_title('u')
ax21.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")
ax22.plot(ElMat, camHex_input_pos_mat[:,4])
ax22.set_title('v')
ax22.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")
ax23.plot(ElMat, camHex_input_pos_mat[:,5])
ax23.set_title('w')
ax23.set(xlabel='Elevation in deg', ylabel = "Rotation in deg")

#plt.plot(ElMat, m2Hex_input_pos_mat)
plt.show()

In [ ]:
mode = await camHex.evt_compensationMode.aget(timeout=10)
print(mode)

In [ ]:
offsetValue = await camHex.evt_compensatedPosition.aget(timeout=30)
print(offsetValue)

In [ ]:
LogMessages = await camHex.evt_logMessage.aget(timeout=10)
print(LogMessages)

In [ ]:
print(camHex.salinfo.event_names)

### Questions 
Does the Hexapod tracks as soon as I send a pointing value?
How do you get the hexapod to apply the lookup table as a function of the elevation?

## Stop the system: stop the pointing kernel?

In [ ]:
await mtptg.cmd_stopTracking.start(timeout=10.)
#send the telescope to Park. Does not exist yet

In [ ]:
await rot.cmd_stop.start(timeout=10.)